In [ ]:
import os

import SimpleITK as sitk
import six
from radiomics import featureextractor, getTestCase
import pandas as pd
import glob
import seaborn as sns

In [ ]:
import tqdm
import numpy as np

In [ ]:
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline


from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split,StratifiedKFold
from scipy.stats import uniform, randint


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
from missforest import MissForest

# first create dict with mask keys and value img

In [ ]:
path_training_msks_auto = glob.glob("train_dataset\auto_generated_mask\*.mha")

In [ ]:
path_testing_masks_auto = glob.glob("test_dataset\auto_generated_mask\*.mha")

In [ ]:
path_external_masks_auto = glob.glob("external_dataset\auto_generated_mask\*.mha")

In [ ]:
path_true_msks = glob.glob("Breast segmentation\MHAs\*\*\*STRUCT1.mha")

In [ ]:
path_images = glob.glob("Breast segmentation\MHAs\*\*\*RECOMBINED1.mha")

In [ ]:
path_images_le = glob.glob("Breast segmentation\MHAs\*\*\*LOW_ENERGY1.mha")

In [ ]:
#for auto prediction
dict_paths_training_auto = {}
empty_msk = []
for temp_mask in tqdm.tqdm(path_training_msks_auto):
    key_path = temp_mask.split("\\")[-1][:-21]
    pos = key_path.split("_")[2]
    view = key_path.split("_")[-1]
    tag_pat = key_path[:9]
    for temp_mha in path_true_msks:
        small_path_mha = temp_mha.split("\\")[-1]
        if tag_pat in small_path_mha and view+ "_" +pos in small_path_mha:
            dict_paths_training_auto[temp_mask]=temp_mha
            continue


In [ ]:
dict_paths_training_auto

In [ ]:
list(dict_paths_training_auto.keys())[0].split("\\")[-1].split("_")

In [ ]:
all_train_jpg = glob.glob("colored_to_jpg\*.jpg")

In [ ]:
train_patients= [temp.split("\\")[-1][:9] for temp in all_train_jpg]

In [ ]:
list_mha_train = [temp for temp in path_true_msks if temp.split("\\")[-3] in train_patients]

In [ ]:
#create list of all the predictions associated to one mha
path_masks_training_auto_copy = path_training_msks_auto.copy()
dict_pred = {}
path_masks_auto = list(dict_paths_training_auto.keys())
for temp_ground_truth in tqdm.tqdm(list_mha_train):
    list_pat_auto = []
    for temp_mask in path_masks_training_auto_copy:
        list_temp = temp_mask.split("\\")[-1].split("_")
        patient_id = list_temp[0] + "_" + list_temp[1]
        pos = list_temp[2]
        view = list_temp[3]
        if patient_id+"_" + view + "_" + pos in temp_ground_truth:
            list_pat_auto.append(temp_mask)
            path_masks_training_auto_copy.remove(temp_mask)
    dict_pred[temp_ground_truth] = list_pat_auto

In [ ]:
train_csv = pd.read_csv("train_set.csv",names=["image_path", "x1", "y1", "x2","y2","outcome"])


In [ ]:
train_img_path = list(train_csv['image_path'])

In [ ]:
# import pickle
# with open(r'E:\ManonData\keras-retinanet-master\auto_generated_labels.pck', 'wb') as handle:
#     pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

## create dict with image and true masks

In [ ]:
df_info_train = pd.read_excel('all_info_train_dataset.xlsx',index=True)
path_train_imgs = list(df_info_train['path_recombined_img'])
path_true_train_masks = list(df_info_train['path_mask_img'])
outcome = list(df_info_train['outcome'])
path_to_match = list(df_info_train['Unnamed: 0'])
colored_imgs_path = os.listdir("colored_to_jpg")
small_colored_imgs_path = [temp[:-4] for temp in colored_imgs_path]
dict_paths_true_train = {}
true_train_outcome = []
for i,temp_mask in enumerate(path_true_train_masks):
    if path_to_match[i] in small_colored_imgs_path:
        dict_paths_true_train[temp_mask] = path_train_imgs[i]
        true_train_outcome.append(outcome[i])

In [ ]:
plt.hist(true_train_outcome)

In [ ]:
df_info_test = pd.read_excel('all_info_test_dataset.xlsx',index=True)
path_test_imgs = list(df_info_test['path_recombined_img'])
path_true_test_masks = list(df_info_test['path_mask_img'])
outcome = list(df_info_test['outcome'])
path_to_match = list(df_info_test['Unnamed: 0'])
colored_imgs_path = os.listdir("colored_to_jpg")
small_colored_imgs_path = [temp[:-4] for temp in colored_imgs_path]
dict_paths_true_test = {}
true_test_outcome = []
for i,temp_mask in enumerate(path_true_test_masks):
    if path_to_match[i] in small_colored_imgs_path:
        dict_paths_true_test[temp_mask] = path_test_imgs[i]
        true_test_outcome.append(outcome[i])

In [ ]:
plt.hist(true_test_outcome)

In [ ]:
df_info_external = pd.read_excel("all_info_external_dataset.xlsx",index=True)
path_external_imgs = list(df_info_external['path_recombined_img'])
path_true_external_masks = list(df_info_external['path_mask_img'])
outcome = list(df_info_external['outcome'])
path_to_match = list(df_info_external['Unnamed: 0'])
colored_imgs_path = os.listdir("colored_to_jpg")
small_colored_imgs_path = [temp[:-4] for temp in colored_imgs_path]
dict_paths_true_external = {}
true_external_outcome = []
for i,temp_mask in enumerate(path_true_external_masks):
    if path_to_match[i] in small_colored_imgs_path:
        dict_paths_true_external[temp_mask] = path_external_imgs[i]
        true_external_outcome.append(outcome[i])

## create dict with image and auto masks

### for the training dataset

In [ ]:
train_csv = "annotations_train_dataset.csv"
df_train = pd.read_csv(train_csv,header=None)

In [ ]:
#labels = list(df_train[5])
key_struct = [elmt.split("\\")[-1][:-4] for elmt in df_train[0]]
reordered_key_struct = [elmt[:10]+elmt.split("_")[-1] +"_"+ elmt[10] for elmt in key_struct]

In [ ]:
import pickle
path = "dict_prediction_train.pkl"
with open(path, 'rb') as file:
    try:
        while True:
            dict_prediction_train =  pickle.load(file)
    except EOFError:
        pass


In [ ]:
dict_paths_auto_mask_train = {}
list_labels_auto_train=[]
for temp_mask in tqdm.tqdm(path_training_msks_auto):
    #key_path = temp_mask.split("\\")[-1][:-12]
    tag_pat = temp_mask.split("\\")[-1][:9]+"_"+temp_mask.split("\\")[-1][:-4].split("_")[3]+"_"+temp_mask.split("\\")[-1][:-4].split("_")[2] 
    for temp_mha in path_images:
        small_path_mha = temp_mha.split("\\")[-1]
        if tag_pat in small_path_mha and tag_pat in reordered_key_struct :
            dict_paths_auto_mask_train[temp_mask]=temp_mha
            for key in list(dict_prediction_train.keys()):
                matching_path = key.split("\\")[-1].split(".")[0] + "_" +key.split("\\")[-1][-1] + "_auto_rt_struct.mha"
                if matching_path in temp_mask and dict_prediction_train[key][-1]*dict_prediction_train[key][1]>0.1: #iou more than 0.1 is considered correct
                    list_labels_auto_train.append(dict_prediction_train[key][-1])
                    continue
                elif matching_path in temp_mask and dict_prediction_train[key][-1]*dict_prediction_train[key][1]<=0.1:
                    list_labels_auto_train.append(0)
                    continue
            #list_labels_auto_train.append(labels[reordered_key_struct.index(tag_pat)])
            continue

### for the testing dataset

In [ ]:
test_csv = "annotations_test_dataset.csv"
df_test = pd.read_csv(test_csv,header=None)

In [ ]:
key_struct = [elmt.split("\\")[-1][:-4] for elmt in df_test[0]]
reordered_key_struct = [elmt[:10]+elmt.split("_")[-1] +"_"+ elmt[10] for elmt in key_struct]

In [ ]:
dict_prediction_test

In [ ]:
import pickle
path = "dict_prediction_test.pkl"
with open(path, 'rb') as file:
    try:
        while True:
            dict_prediction_test =  pickle.load(file)
    except EOFError:
        pass

In [ ]:
dict_paths_auto_mask_test = {}
list_labels_auto_test=[]
for temp_mask in tqdm.tqdm(path_testing_masks_auto):
    #key_path = temp_mask.split("\\")[-1][:-12]
    tag_pat = temp_mask.split("\\")[-1][:9]+"_"+temp_mask.split("\\")[-1][:-4].split("_")[3]+"_"+temp_mask.split("\\")[-1][:-4].split("_")[2] 
    for temp_mha in path_images:
        small_path_mha = temp_mha.split("\\")[-1]
        if tag_pat in small_path_mha and tag_pat in reordered_key_struct :
            dict_paths_auto_mask_test[temp_mask]=temp_mha
            for key in list(dict_prediction_test.keys()):
                matching_path = key.split("\\")[-1].split(".")[0] + "_" +key.split("\\")[-1][-1] + "_auto_rt_struct.mha"
                if matching_path in temp_mask and dict_prediction_test[key][-1]*dict_prediction_test[key][1]>0.1: #iou more than 0.1 is considered correct
                    list_labels_auto_test.append(dict_prediction_test[key][-1])
                    continue
                elif matching_path in temp_mask and dict_prediction_test[key][-1]*dict_prediction_test[key][1]<=0.1:
                    list_labels_auto_test.append(0)
                    continue
            #list_labels_auto_test.append(labels[reordered_key_struct.index(tag_pat)])
            continue

In [ ]:
dict_le_re = {}
for temp_mha in path_images:
    parent = os.path.dirname(temp_mha)
    for temp_mha_le in path_images_le:
        if parent in temp_mha_le:
            dict_le_re[temp_mha]=temp_mha_le

In [ ]:
list_patient = []
for elmt in list(dict_paths_auto_mask_test.keys()):
    list_patient.append(elmt[:9])

In [ ]:
all_patients = np.unique(list_patient)

In [ ]:
df_cl_features  = pd.read_excel(r"E:\ManonData\keras-retinanet-master\Clinical_Features_all.xlsx")

In [ ]:
filtered_dict = {}
for pat in all_patients:
    filtered_dict[pat] = list(df_cl_features[['Outcome','specificity']][df_cl_features['UM_ID']==pat].values[0])

In [ ]:
filtered_df = pd.DataFrame.from_dict(filtered_dict,columns = ['outcome','specificity'],orient='index')

In [ ]:
#filtered_df.to_excel(r'E:\ManonData\keras-retinanet-master\all_datasets\test_dataset_experiment\data for sergey\outcome_specificity_per_patient.xlsx')

In [ ]:
bbox_df = pd.read_excel(r'E:\ManonData\keras-retinanet-master\all_datasets\test_dataset_experiment\data for sergey\data_bbox.xlsx')

In [ ]:
from skimage.measure import regionprops

list_paths = list(bbox_df["Unnamed: 0"])
dict_temp = {}
main_path = r"E:\ManonData\keras-retinanet-master\all_datasets\test_dataset_experiment\auto_generated_mask\\"
for path in tqdm.tqdm(list_paths):
    temp_path = main_path + path
    temp_image = sitk.GetArrayFromImage(sitk.ReadImage(temp_path))
    props = regionprops(temp_image)
    r0, c0, r1, c1 = props[0].bbox
    dict_temp[path] = [r0, c0, r1, c1]

In [ ]:
df_for_sergey = pd.DataFrame.from_dict(dict_temp,columns=['x_min','y_min','x_max','y_max'],orient='index')


In [ ]:
bbox_df[['x_min','y_min','x_max','y_max']] = df_for_sergey

In [ ]:
bbox_df=bbox_df.set_index("Unnamed: 0")

In [ ]:
bbox_df

In [ ]:
import shutil
#for sergey
dict_paths_auto_mask_test = {}
for temp_mask in tqdm.tqdm(path_testing_masks_auto):
    #key_path = temp_mask.split("\\")[-1][:-12]
    tag_pat = temp_mask.split("\\")[-1][:9]+"_"+temp_mask.split("\\")[-1][:-4].split("_")[3]+"_"+temp_mask.split("\\")[-1][:-4].split("_")[2] 
    for temp_mha in path_images:
        small_path_mha = temp_mha.split("\\")[-1]
        if tag_pat in small_path_mha and tag_pat in reordered_key_struct :
            temp_mha_le = dict_le_re[temp_mha]
#             shutil.copy(temp_mha_le, r"E:\ManonData\keras-retinanet-master\all_datasets\test_dataset_experiment\data for sergey\\")
#             shutil.copy(temp_mha, r"E:\ManonData\keras-retinanet-master\all_datasets\test_dataset_experiment\data for sergey\\")
            for key in list(dict_prediction_test.keys()):
                matching_path = key.split("\\")[-1].split(".")[0] + "_" +key.split("\\")[-1][-1] + "_auto_rt_struct.mha"
                if matching_path in temp_mask and dict_prediction_test[key][1]>0.1: #iou more than 0.1 is considered correct
                    dict_paths_auto_mask_test[temp_mask.split("\\")[-1]]=[small_path_mha,temp_mha_le.split("\\")[-1]]+list(dict_prediction_test[key][0])+[dict_prediction_test[key][1],1,dict_prediction_test[key][-1]]
                    continue
                elif matching_path in temp_mask and dict_prediction_test[key][1]<=0.1:
                    dict_paths_auto_mask_test[temp_mask.split("\\")[-1]]=[small_path_mha,temp_mha_le.split("\\")[-1]]+list(dict_prediction_test[key][0])+[dict_prediction_test[key][1],0,'na']
                    continue
            #list_labels_auto_test.append(labels[reordered_key_struct.index(tag_pat)])
            continue

In [ ]:
dict_paths_auto_mask_test

## load extraction parameters

In [ ]:
paramsFile=r"E:\ManonData\keras-retinanet-master\MR_2D_extraction.yaml"

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor(paramsFile,shape2D=True,force2D=True,force2Ddimension=True,resampledPixelSpacing=None)

In [ ]:
#set to all the features

extractor.addProvenance(False)
extractor.disableAllFeatures()
extractor.enableImageTypes(Original={})

extractor.enableFeatureClassByName('firstorder', enabled=True)
#extractor.enableFeatureClassByName('shape', enabled=False)
extractor.enableFeatureClassByName('shape2D', enabled=True)
extractor.enableFeatureClassByName('glcm', enabled=True)
extractor.enableFeatureClassByName('glrlm', enabled=True)
extractor.enableFeatureClassByName('glszm', enabled=True)
extractor.enableFeatureClassByName('gldm', enabled=True)
extractor.enableFeatureClassByName('ngtdm', enabled=True)

## create feature tables

In [ ]:
def generate_features_table(dict_paths):
    #initialize table
    featureVector = extractor.execute(list(dict_paths.values())[0],list(dict_paths.keys())[0])
    temp_dataset =pd.Series(featureVector)
    feature_df =pd.DataFrame([temp_dataset],columns=list(featureVector.keys()),index = [list(dict_paths.keys())[0]] )
    for temp_mask in tqdm.tqdm(list(dict_paths.keys())[1:]):
        featureVector = extractor.execute(dict_paths[temp_mask], temp_mask)
        temp_dataset =pd.Series(featureVector)
        feature_df.loc[temp_mask]=temp_dataset.values
    return feature_df

In [ ]:
df_true_mask_train_features = generate_features_table(dict_paths_true_train)
#df_true_mask_train_features.to_csv(r"E:\ManonData\keras-retinanet-master\all_datasets\train_dataset_sergey\train_features_true_mask.csv")

In [ ]:
df_true_mask_test_features = generate_features_table(dict_paths_true_test)


In [ ]:
df_true_mask_test_features=pd.read_csv(r"E:\ManonData\keras-retinanet-master\all_datasets\test_dataset_experiment\test_features_true_mask.csv")

In [ ]:
df_auto_mask_train_features = generate_features_table(dict_paths_auto_mask_train)


In [ ]:
df_auto_mask_test_features = generate_features_table(dict_paths_auto_mask_test)


In [ ]:
df_true_mask_external_features = generate_features_table(dict_paths_true_external)

In [ ]:
df_auto_mask_external_features = generate_features_table(dict_paths_auto_mask_external)


## ML model and results

### pre-processing for auto masks

In [ ]:
df_auto_mask_train_features['labels']=list_labels_auto_train

In [ ]:
def sampling_k_elements(group, k=500):
    if len(group) < k:
        return group
    return group.sample(k)

balanced_dataset = df_auto_mask_train_features.groupby('labels').apply(sampling_k_elements).reset_index(drop=True)

In [ ]:
balanced_dataset = balanced_dataset.sample(frac=1).reset_index(drop=True)

In [ ]:
plt.hist(balanced_dataset['labels'])

In [ ]:
balanced_labels = balanced_dataset['labels']

In [ ]:
balanced_dataset=balanced_dataset.drop(columns = 'labels')

In [ ]:
columns = list(balanced_dataset.columns)
for idx,temp_type in enumerate(list(balanced_dataset.dtypes)):
    if temp_type == 'O':
        balanced_dataset[columns[idx]] = balanced_dataset[columns[idx]].astype(float)

In [ ]:
cor = balanced_dataset.corr('spearman').abs()
upper_tri = cor.where(np.triu(np.ones(cor.shape),k=1).astype(np.bool))
to_drop = []
for column in tqdm.tqdm(upper_tri.columns):
    for row in upper_tri.columns:
        if abs(upper_tri[column][row])>0.85:
            if np.sum(abs(upper_tri[column])) > np.sum(abs(upper_tri[row])):
                to_drop.append(column)
            else:
                to_drop.append(row)

In [ ]:
decor_dataset_auto_test = df_auto_mask_test_features.drop(to_drop,axis=1)

In [ ]:
decor_dataset_auto_test.reset_index(drop=True,inplace=True)

In [ ]:
decor_dataset_auto = balanced_dataset.drop(to_drop, axis=1)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold = 0.01)
selector.fit(decor_dataset_auto)
thres_dataset_auto = decor_dataset_auto.loc[:, selector.get_support()]


In [ ]:
thres_dataset_auto_test = decor_dataset_auto_test.loc[:, selector.get_support()]

In [ ]:
thres_dataset_auto_test =thres_dataset_auto_test.astype(float)

In [ ]:
thres_dataset_auto.shape

## for true contours

In [ ]:
# cor = df_true_mask_train_features.corr('spearman').abs()
# upper_tri = cor.where(np.triu(np.ones(cor.shape),k=1).astype(np.bool))
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)] #feature with the highest average correlation to the remaining features was removed.
# to_drop

In [ ]:
## z-score first and remove maybe the ones wich are not normaly distributed

## for radiomics only

In [ ]:
#remove patient infos
img_paths = df_true_mask_train_features["Unnamed: 0"]
img_paths_test = df_true_mask_test_features["Unnamed: 0"]
df_true_mask_train_features =df_true_mask_train_features.drop(["Unnamed: 0"], axis=1)
df_true_mask_test_features=df_true_mask_test_features.drop(["Unnamed: 0"], axis=1)

In [ ]:
img_paths_external = df_true_mask_external_features["Unnamed: 0"]
df_true_mask_external_features=df_true_mask_external_features.drop(["Unnamed: 0"], axis=1)

### eliminate non-normal features

In [ ]:
# from scipy import stats ##not a necessary step for xgboost but mandatory for SVM
# alpha = 1e-3
# non_normal_col = []
# for col in tqdm.tqdm(list(df_true_mask_train_features.columns)[1:]):
#     k2, p = stats.normaltest(df_true_mask_train_features[col])
#     if p > alpha: 
#         print(col)
#         non_normal_col.append(col)

In [ ]:
#df_true_mask_train_normal= df_true_mask_train_features.drop(non_normal_col, axis=1)

In [ ]:
#df_true_mask_test_normal= df_true_mask_test_features.drop(non_normal_col, axis=1)

In [ ]:
##z-score
from scipy.stats import zscore
mean_std={}
for var in df_true_mask_train_features.columns:
    temp_mean = df_true_mask_train_features[var].mean()
    temp_std = df_true_mask_train_features[var].std()
    mean_std[var]=(temp_mean,temp_std)
    df_true_mask_train_features[var] = (df_true_mask_train_features[var]-temp_mean)/temp_std

In [ ]:
##applied to test set
for var in df_true_mask_test_features.columns:
    df_true_mask_test_features[var] = (df_true_mask_test_features[var]-mean_std[var][0])/mean_std[var][1]

In [ ]:
##applied to external set
for var in df_true_mask_external_features.columns:
    df_true_mask_external_features[var] = (df_true_mask_external_features[var]-mean_std[var][0])/mean_std[var][1]

In [ ]:
##variance threshold
from sklearn.feature_selection import VarianceThreshold 
selector = VarianceThreshold(threshold = 0.01)
selector.fit(df_true_mask_train_features)
thres_dataset_train = df_true_mask_train_features.loc[:, selector.get_support()]
thres_dataset_test = df_true_mask_test_features.loc[:, selector.get_support()]
thres_dataset_external = df_true_mask_external_features.loc[:, selector.get_support()]

In [ ]:
len(selector.get_support())

In [ ]:
cor = thres_dataset_train.corr('spearman').abs()
upper_tri = cor.where(np.triu(np.ones(cor.shape),k=1).astype(np.bool))

In [ ]:
to_drop = []
for column in upper_tri.columns:
    for row in upper_tri.columns:
        if upper_tri[column][row]>0.85:
            if np.sum(upper_tri[column]) > np.sum(upper_tri[row]): #need to check equivalent function in R cor findcorrelation
                to_drop.append(column)
            else:
                to_drop.append(row)

In [ ]:
to_drop = np.unique(to_drop)

In [ ]:
decor_dataset = thres_dataset_train.drop(to_drop, axis=1)

In [ ]:
#with clinical features
decor_dataset = pd.concat([decor_dataset,clinical_train], axis=1)

In [ ]:
decor_dataset_test = thres_dataset_test.drop(to_drop, axis=1)

In [ ]:
decor_dataset_test = pd.concat([decor_dataset_test,clinical_test], axis=1)

In [ ]:
decor_dataset_external = thres_dataset_external.drop(to_drop, axis=1)

In [ ]:
len(decor_dataset.columns)

In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False, colsample_bytree=1,
  objective= 'binary:logistic',eval_metric = 'logloss', nthread=4, scale_pos_weight=1, seed=27)

In [ ]:
min_features_to_select = 1  # Minimum number of features to consider
rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(10),
              scoring='roc_auc',
              min_features_to_select=min_features_to_select)
rfecv.fit(decor_dataset, true_train_outcome)
plt.figure(figsize=(8, 6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

In [ ]:
from sklearn.feature_selection import RFECV
# min_features_to_select = 1  # Minimum number of features to consider
# rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(10),
#               scoring='roc_auc',
#               min_features_to_select=min_features_to_select)
# rfecv.fit(decor_dataset, true_train_outcome)
plt.figure(figsize=(8, 6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

In [ ]:
#xgb.__version__ former version '1.0.0-SNAPSHOT'

In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False, colsample_bytree=1,
  objective= 'binary:logistic',eval_metric = 'logloss', nthread=4, scale_pos_weight=1, seed=27)

In [ ]:
rfe = RFE(estimator=model,n_features_to_select=8, random_state=0)
#rfe = RFE(estimator=RandomForestClassifier(),n_features_to_select=10)
n8_features_train_set = rfe.fit_transform(decor_dataset, true_train_outcome)
filtered_col = np.extract(rfe.support_, np.array(decor_dataset.columns))
n8_features_train_set = pd.DataFrame(data=n8_features_train_set, columns=filtered_col)

In [ ]:
filtered_col

In [ ]:
filtered_col

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 20, num = 10)]
max_depth.append(None)
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid

# param_test_rf = {
#     'rfe__n_features_to_select':[8,10,12],
#     'model__n_estimators': n_estimators,
#     'model__max_features': max_features,
#     'model__max_depth': max_depth,
#     'model__min_samples_leaf': min_samples_leaf}
param_test_rf = {
    'n_estimators': n_estimators,
#    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap':bootstrap
}

# param_test_xgb = {
# 'rfe__n_features_to_select':[6,8,10,12],
# 'xgb__max_depth':range(2,4,1),
# 'xgb__min_child_weight':range(1,6,2),
# 'xgb__n_estimators': range(60, 220, 40),
# 'xgb__gamma':[i*0.1 for i in range(3,10)],
# 'xgb__learning_rate':[10**(-i) for i in range(3,5)]
# #'xgb__weights' : [ 10, 50, 75, 99]
# }

param_test_xgb = {
'max_depth':range(2,4,1),
'min_child_weight':range(1,6,2),
'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)],
'gamma':[i*0.1 for i in range(3,10)],
'learning_rate':[10**(-i) for i in range(2,7)]
#'xgb__weights' : [ 10, 50, 75, 99]
}


kfold = StratifiedKFold(n_splits=10, random_state=42)





#model = RandomForestClassifier()

pipeline = Pipeline(steps=[('rfe',rfe),('model',model)])

# gsearch1 = GridSearchCV(pipeline, 
#  param_grid = param_test_rf, scoring='roc_auc',n_jobs=12,iid=False, cv=kfold,verbose=1)

# gsearch1.fit(n10_features_train_set, true_train_outcome)
gsearch1 = GridSearchCV(model, 
  param_grid = param_test_xgb, scoring='roc_auc',n_jobs=4,iid=False, cv=kfold,verbose=1)

gsearch1.fit(n8_features_train_set, true_train_outcome)

In [ ]:
n8_features_test_set = rfe.transform(decor_dataset_test)
#n8_features_external_set = rfe.transform(decor_dataset_external)

In [ ]:
n8_features_test_set = pd.DataFrame(data=n8_features_test_set, columns=filtered_col)
#n8_features_external_set = pd.DataFrame(data=n8_features_external_set, columns=filtered_col)

In [ ]:
import pickle

#
# Create your model here (same as above)
#
# tuple_objects = (gsearch1,filtered_col,n8_features_train_set,n8_features_test_set)
# # Save to file in the current working directory
# pkl_filename = r"E:\ManonData\keras-retinanet-master\examples\pickle_model_best_xgb_true_labels.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(tuple_objects, file)

# # Load from file
# with open(pkl_filename, 'rb') as file:
#     gsearch1,filtered_col,n8_features_train_set,n8_features_test_set = pickle.load(file)

In [ ]:
#rf
# {'bootstrap': True,
#  'max_depth': 16,
#  'min_samples_leaf': 2,
#  'n_estimators': 670}

In [ ]:
#for radiomics only
# gsearch1.best_params_
# {'gamma': 0.6000000000000001,
#  'learning_rate': 0.01,
#  'max_depth': 3,
#  'min_child_weight': 1,
#  'n_estimators': 560}

In [ ]:
#for radiomics and clinical features
# gsearch1.best_params_
# {'gamma': 0.8,
#  'learning_rate': 0.01,
#  'max_depth': 3,
#  'min_child_weight': 3,
#  'n_estimators': 890}

In [ ]:
##for the automated segmentation
kfold = KFold(n_splits=10, random_state=42)

gsearch2 = GridSearchCV(estimator = xgb.XGBClassifier( colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=8,iid=False, cv=kfold,verbose=1)

gsearch2.fit(thres_dataset_auto, list(balanced_labels))

In [ ]:
gsearch1.best_score_ #0.8673408532469921 for ground truth with 10 features #0.8942350794061321 with no feature selection 0.9167289576454515 with 8 features

In [ ]:
gsearch2.best_score_ 

In [ ]:
gsearch1.best_params_

In [ ]:
best_estimator = gsearch1.best_estimator_
best_estimator.feature_names = filtered_col

In [ ]:
best_estimator.feature_names

In [ ]:
best_estimator.feature_importances_

In [ ]:
best_estimator_auto = gsearch2.best_estimator_

In [ ]:
plt.rcParams["figure.figsize"] = (10,20)
xgb.plot_importance(best_estimator.get_booster()) #to double check
plt.show()

In [ ]:
proba_train=best_estimator.predict_proba(n8_features_train_set)
proba_test=best_estimator.predict_proba(n8_features_test_set)
#proba_external=best_estimator.predict_proba(n8_features_external_set)

In [ ]:
proba_train_auto=best_estimator_auto.predict_proba(thres_dataset_auto)
proba_test_auto=best_estimator_auto.predict_proba(thres_dataset_auto_test)

In [ ]:
true_external_binary_outcome = []
for elmt in true_external_outcome:
    if elmt == 1 or elmt == 2 or elmt == 3:
        true_external_binary_outcome.append(1)
    else:
        true_external_binary_outcome.append(0)

In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False, colsample_bytree=1,
  objective= 'binary:logistic',eval_metric = 'logloss', nthread=4, scale_pos_weight=1, seed=27,gamma=0.6,learning_rate=0.01, max_depth=3,min_child_weight=1,n_estimators=560)


In [ ]:
#also test the full test dataset and the external dataset here
cv = StratifiedKFold(n_splits=10,random_state =42)



tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

tprs_test = []
aucs_test = []
mean_fpr_test = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(8,6))
for i, (train, test) in enumerate(cv.split(n8_features_train_set, true_train_outcome)):
    model.fit(np.array(n8_features_train_set)[train], np.array(true_train_outcome)[train])
#     viz = plot_roc_curve(model, np.array(n8_features_train_set)[test], np.array(true_train_outcome)[test],
#                          name='ROC fold {}'.format(i),
#                          alpha=0.3, lw=1, ax=ax)
    proba_test =  model.predict_proba(np.array(n8_features_train_set)[test])
    fpr_test, tpr_test, thresholds_test = sklearn.metrics.roc_curve( np.array(true_train_outcome)[test],np.array(proba_test[:,1]))
    interp_tpr = np.interp(mean_fpr, fpr_test, tpr_test)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    roc_auc_test = sklearn.metrics.auc(fpr_test, tpr_test)
    aucs.append(roc_auc_test)
    
    proba_test_full =  model.predict_proba(np.array(n8_features_test_set))
    fpr_test_full, tpr_test_full, thresholds_test_full = sklearn.metrics.roc_curve( np.array(true_test_outcome),np.array(proba_test_full[:,1]))
    interp_tpr_test = np.interp(mean_fpr_test, fpr_test_full, tpr_test_full)
    interp_tpr_test[0] = 0.0
    tprs_test.append(interp_tpr_test)
    roc_auc_test_full = sklearn.metrics.auc(fpr_test_full, tpr_test_full)
    aucs_test.append(roc_auc_test_full)
    
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = sklearn.metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC test (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.3,
                label=r'$\pm$ 1 std. dev.')

# mean_tpr_test = np.mean(tprs_test, axis=0)
# mean_tpr_test[-1] = 1.0
# mean_auc_test = sklearn.metrics.auc(mean_fpr_test, mean_tpr_test)
# std_auc_test = np.std(aucs_test)
# ax.plot(mean_fpr_test, mean_tpr_test, color='g',
#         label=r'Mean ROC validation (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc_test, std_auc_test),
#         lw=2, alpha=.8)

# std_tpr_test = np.std(tprs_test, axis=0)
# tprs_upper = np.minimum(mean_tpr_test + std_tpr_test, 1)
# tprs_lower = np.maximum(mean_tpr_test - std_tpr_test, 0)
# ax.fill_between(mean_fpr_test, tprs_lower, tprs_upper, color='green', alpha=.3,
#                 label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax.legend(loc="lower right")
plt.show()

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score
np.random.seed(32)
def bootstrap(x, f, nsamples=1000):
    stats = [f(x[np.random.randint(x.shape[0], size=x.shape[0])]) for _ in range(nsamples)]
    return np.percentile(stats, (2.5, 97.5))

#def bootstrap_auc(clf, X_train, y_train, X_test, y_test,X_external,y_external, nsamples=1000):
clf, X_train, y_train, X_test, y_test,X_external,y_external = model,np.array(n8_features_train_set), np.array(true_train_outcome), np.array(n8_features_test_set), np.array(true_test_outcome),np.array(n8_features_external_set),np.array(true_external_binary_outcome)
clf.fit(X_train, y_train)
nsamples=1000
auc_values = []
auc_values_external=[]

accuracy_values = []
accuracy_values_external=[]

precision_values = []
precision_values_external=[]

recall_values = []
recall_values_external=[]

tprs = []
mean_fpr = np.linspace(0, 1, 100)

tprs_external = []
mean_fpr_external = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(8,6))
for b in tqdm.tqdm(range(nsamples)):
    idx_test = np.random.randint(X_test.shape[0], size=X_test.shape[0])
    idx_external = np.random.randint(X_external.shape[0], size=X_external.shape[0])
    pred = clf.predict_proba(X_test[idx_test])[:, 1]
    accuracy_values.append(accuracy_score(y_test[idx_test],(np.array(pred)>0.5).astype(int)))
    precision_values.append(precision_score(y_test[idx_test],(np.array(pred)>0.5).astype(int)))
    recall_values.append(recall_score(y_test[idx_test],(np.array(pred)>0.5).astype(int)))
                         
    pred_external= clf.predict_proba(X_external[idx_external])[:, 1]
    accuracy_values_external.append(accuracy_score(y_external[idx_external],(np.array(pred_external)>0.5).astype(int)))
    precision_values_external.append(precision_score(y_external[idx_external],(np.array(pred_external)>0.5).astype(int)))
    recall_values_external.append(recall_score(y_external[idx_external],(np.array(pred_external)>0.5).astype(int)))                                             

    fpr_test, tpr_test, thresholds_test = sklearn.metrics.roc_curve( y_test[idx_test], pred)
    interp_tpr = np.interp(mean_fpr, fpr_test, tpr_test)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)

    fpr_external, tpr_external, thresholds_external = sklearn.metrics.roc_curve(y_external[idx_external], pred_external)
    interp_tpr_external = np.interp(mean_fpr_external, fpr_external, tpr_external)
    interp_tpr_external[0] = 0.0
    tprs_external.append(interp_tpr_external)

    roc_auc = sklearn.metrics.auc(fpr_test, tpr_test)
    roc_auc_external = sklearn.metrics.auc(fpr_external, tpr_external)
    auc_values.append(roc_auc)
    auc_values_external.append(roc_auc_external)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
    label='Chance', alpha=.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = sklearn.metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(auc_values)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC test (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.3,
                label=r'$\pm$ 1 std. dev.')

mean_tpr_external = np.mean(tprs_external, axis=0)
mean_tpr_external[-1] = 1.0
mean_auc_external = sklearn.metrics.auc(mean_fpr_external, mean_tpr_external)
std_auc_external = np.std(auc_values_external)
ax.plot(mean_fpr_external, mean_tpr_external, color='g',
        label=r'Mean ROC external dataset (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc_external, std_auc_external),
        lw=2, alpha=.8)

std_tpr_external = np.std(tprs_external, axis=0)
tprs_upper = np.minimum(mean_tpr_external + std_tpr_external, 1)
tprs_lower = np.maximum(mean_tpr_external - std_tpr_external, 0)
ax.fill_between(mean_fpr_external, tprs_lower, tprs_upper, color='green', alpha=.3,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax.legend(loc="lower right")
plt.show()
print(np.percentile(auc_values, (2.5, 97.5)),np.percentile(auc_values_external, (2.5, 97.5)))
print(np.mean(precision_values),np.percentile(precision_values, (2.5, 97.5)),np.mean(precision_values_external),np.percentile(precision_values_external, (2.5, 97.5)))

In [ ]:
print(np.mean(accuracy_values),np.percentile(accuracy_values, (2.5, 97.5)),np.mean(accuracy_values_external),np.percentile(accuracy_values_external, (2.5, 97.5)))

In [ ]:
print(np.mean(recall_values),np.percentile(recall_values, (2.5, 97.5)),np.mean(recall_values_external),np.percentile(recall_values_external, (2.5, 97.5)))

In [ ]:
import sklearn
from matplotlib import pyplot as plt

fpr_train, tpr_train, thresholds_test = sklearn.metrics.roc_curve(true_train_outcome,np.array(proba_train[:,1]))
roc_auc_train = sklearn.metrics.auc(fpr_train, tpr_train)
fpr_test, tpr_test, thresholds_test = sklearn.metrics.roc_curve(true_test_outcome,np.array(proba_test[:,1]))
roc_auc_test = sklearn.metrics.auc(fpr_test, tpr_test)
# fpr_external, tpr_external, thresholds_external = sklearn.metrics.roc_curve(true_external_binary_outcome,np.array(proba_external[:,1]))
# roc_auc_external = sklearn.metrics.auc(fpr_external, tpr_external)

plt.figure(figsize=(8, 6))
plt.rcParams['font.size'] = '14'
lw = 2
plt.plot(fpr_train, tpr_train,
         label='ROC curve train dataset (area = {0:0.2f})'
               ''.format(roc_auc_train),
         color='darkorange', linewidth=2)
plt.plot(fpr_test, tpr_test,
         label='ROC curve test dataset (area = {0:0.2f})'
               ''.format(roc_auc_test),
         color='darkblue', linewidth=2)
# plt.plot(fpr_external, tpr_external,
#          label='ROC curve external dataset (area = {0:0.2f})'
#                ''.format(roc_auc_external),
#          color='green', linewidth=2)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic LN vs no LN')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import classification_report #recall is sensitivity
print(classification_report(true_test_outcome,(np.array(proba_test[:,1])>0.5).astype(int), target_names=["benign","malignant"]))

In [ ]:
print(classification_report(true_external_binary_outcome,(np.array(proba_external[:,1])>0.5).astype(int), target_names=["benign","malignant"]))

In [ ]:
from sklearn.metrics import confusion_matrix
def print_confusion_matrix(true_labels, predicted_labels, class_names, figsize = (6,5), fontsize=14,normalize=True):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    sns.set(font_scale=1.4)
    confusion_matrix= sklearn.metrics.confusion_matrix(true_labels,predicted_labels)
    if normalize:
        confusion_matrix = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        if normalize:
            fmt = '.2f' 
            heatmap = sns.heatmap(df_cm, annot=True, fmt=fmt,cmap="Blues",vmin=0, vmax=1)
        else:
            fmt = 'd' 
            heatmap = sns.heatmap(df_cm, annot=True, fmt=fmt,cmap="Blues",vmax=np.max(confusion_matrix),vmin=0)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fig.tight_layout()
    return fig
      

# Plot non-normalized confusion matrix
fig = print_confusion_matrix(true_labels=true_test_outcome, predicted_labels = (np.array(proba_test[:,1])>0.1).astype(int), class_names=["benign","malignant"])
#print(accuracy_score(y_true_patient,bin_test_pred))

In [ ]:
fpr_train, tpr_train, thresholds_test = sklearn.metrics.roc_curve(list(balanced_labels),np.array(proba_train_auto[:,1]))
roc_auc_train = sklearn.metrics.auc(fpr_train, tpr_train)
fpr_test, tpr_test, thresholds_test = sklearn.metrics.roc_curve(list_labels_auto_test,np.array(proba_test_auto[:,1]))
roc_auc_test = sklearn.metrics.auc(fpr_test, tpr_test)

plt.figure(figsize=(8, 6))
plt.rcParams['font.size'] = '14'
lw = 2
plt.plot(fpr_train, tpr_train,
         label='ROC curve train dataset (area = {0:0.2f})'
               ''.format(roc_auc_train),
         color='darkorange', linewidth=2)
plt.plot(fpr_test, tpr_test,
         label='ROC curve test dataset (area = {0:0.2f})'
               ''.format(roc_auc_test),
         color='darkblue', linewidth=2)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic LN vs no LN')
plt.legend(loc="lower right")
plt.show()

In [ ]:
fig = print_confusion_matrix(true_labels=list_labels_auto_test, predicted_labels = (np.array(proba_test_auto[:,1])>0.5).astype(int), class_names=["benign","malignant"],normalize=False)


In [ ]:
sum(lbl_full_dataset)/len(lbl_full_dataset)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier()
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(thres_dataset, lbl_full_dataset)

In [ ]:
features10_dataset = selector.transform(thres_dataset)